In [8]:
# train_model.py

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Hannanum
import joblib

# 데이터 로드
csv_file_path = 'naver.csv' # 케글 데이터셋 : https://www.kaggle.com/datasets/bmofinnjake/naverwebtoon-datakorean
webtoon_data = pd.read_csv(csv_file_path, encoding='utf-8')

# 불필요한 컬럼 제거
webtoon_data = webtoon_data.drop(['id', 'rating', 'date', 'link', 'free'], axis=1)

# 불용어 정의
stop_words = set(['것', '등', '들', '및', '을', '를', '이', '가'])
stop_words |= set("를 의 웹 툰 웹툰 웹툰판 이번 들 등 수 이 부 판 뿐 여자 남자 그 것 나 그 그녀 속 시작 속".split())
stop_words |= set("이야기 데 전 후 두 앞 뒤 그들 때문 사람 두 신작 한 자신 소년 소녀 만화".split())

# 유사 장르 정의
similar_genres = {
    '로맨스': ['로맨스', '순정', '러브코미디', '감성'],
    '코믹': ['코믹', '개그', '코메디', '일상'],
    '액션': ['액션', '무협', '배틀', '전투'],
    '스릴러': ['스릴러', '미스터리', '공포', '호러', '서스펜스'],
    '판타지': ['판타지', '드라마', '이세계', '마법'],
    '학원': ['학원', '학교', '성장'],
    '스포츠': ['스포츠', '격투'],
    '시대극': ['시대극', '역사'],
    'SF': ['SF', '공상과학'],
    '모험': ['모험', '탐험'],
    '직업': ['직업', '회사', '사회생활'],
    '음식': ['음식', '요리'],
    '동물': ['동물', '펫'],
    '가족': ['가족', '육아'],
    '초자연': ['초자연', '귀신', '괴물'],
    '범죄': ['범죄', '수사'],
    '군대': ['군대', '밀리터리'],
    '착각': ['착각', '오해'],
    '성인': ['성인', '19금'],
}

# 장르 확장 함수
def expand_genres(genre):
    for key, values in similar_genres.items():
        if genre in values:
            return ' '.join(values)
    return genre

# 텍스트 데이터 결합 및 장르 확장
webtoon_data['expanded_genre'] = webtoon_data['genre'].apply(expand_genres)

# NaN 값을 빈 문자열로 대체
webtoon_data = webtoon_data.fillna('')

webtoon_data['combined_features'] = (
    webtoon_data['author'].astype(str) + ' ' +
    webtoon_data['expanded_genre'].astype(str) + ' ' +
    webtoon_data['description'].astype(str) + ' ' +
    webtoon_data['completed'].astype(str) + ' ' +
    webtoon_data['age'].astype(str)
)

# 한국어 전처리 함수
hannanum = Hannanum()

def preprocess_text(text):
    if not isinstance(text, str):
        return ''
    # 명사 추출
    nouns = hannanum.nouns(text)
    # 불용어 제거
    filtered_nouns = [word for word in nouns if word not in stop_words]
    return ' '.join(filtered_nouns)

# 텍스트 전처리 적용
webtoon_data['processed_features'] = webtoon_data['combined_features'].apply(preprocess_text)

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(webtoon_data['processed_features'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 모델 및 데이터 저장
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')
joblib.dump(cosine_sim, 'cosine_sim.joblib')
joblib.dump(similar_genres, 'similar_genres.joblib')
webtoon_data[['title', 'author', 'genre', 'description', 'completed', 'age', 'processed_features']].to_pickle('processed_webtoon_data.pkl')

print("모델 훈련 및 데이터 처리가 완료되었습니다.")
print(f"저장된 데이터 열: {webtoon_data.columns.tolist()}")

모델 훈련 및 데이터 처리가 완료되었습니다.
저장된 데이터 열: ['title', 'author', 'genre', 'description', 'completed', 'age', 'expanded_genre', 'combined_features', 'processed_features']
